# Create an Agent to create insurance premium

The steps to complete this notebook are:

1. Import the needed libraries
1. Create the Knowledge Base for Amazon Bedrock and sync data to Knowledge Base
1. Create the Agent for Amazon Bedrock
1. Test the Agent

## 1. Import the needed libraries and complete the setup

In [6]:
!pip install --upgrade -q opensearch-py
!pip install --upgrade -q requests-aws4auth
!pip install --upgrade -q boto3
!pip install --upgrade -q botocore
!pip install --upgrade -q awscli

In [7]:
import logging
import boto3
import time
import json
import uuid
import pprint
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

In [8]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [9]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
open_search_serverless_client = boto3.client('opensearchserverless')

In [10]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '657506130129')

In [19]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
agent_name = "agent-explorers-insurance-agent"
agent_alias_name = f"{agent_name}-alias"
bucket_name = "agentexplorer-insurance-data"
bucket_arn = f"arn:aws:s3:::{bucket_name}"
bedrock_agent_bedrock_allow_policy_name = f"{agent_name}-bedrock-allow-policy"
bedrock_agent_s3_allow_policy_name = f"{agent_name}-s3-allow-policy"
bedrock_agent_kb_allow_policy_name = f"{agent_name}-kb-allow-policy"
agent_role_name = f'{agent_name}-exec-role'
kb_name = f'{agent_name}-kb'
data_source_name = f'{kb_name}-datasource'
kb_key = 'knowledgebase-docs'
kb_role_name = f'{kb_name}-exec-role'
kb_bedrock_allow_policy_name = f"{kb_name}-bedrock-allow-policy"
kb_aoss_allow_policy_name = f"{kb_name}-aoss-allow-policy"
kb_s3_allow_policy_name = f"{kb_name}-s3-allow-policy"
kb_collection_name = "agent-explorers-insurance-kbc"
# Select Amazon titan as the embedding model
embedding_model_arn = f'arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v1'
kb_vector_index_name = "bedrock-knowledge-base-index"
kb_metadataField = 'bedrock-knowledge-base-metadata'
kb_textField = 'bedrock-knowledge-base-text'
kb_vectorField = 'bedrock-knowledge-base-vector'

# agent configuration
agent_instruction = """
You are an agent that support users working with Amazon Bedrock. You have access to Bedrock's documentation in a Knowledge Base
and you can Answer questions from this documentation. Only answer questions based on the documentation and reply with 
"There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?" 
If the answer to the question is not available in the documentation
"""

# inference_profile = "us.anthropic.claude-3-sonnet-20240229-v1:0"
inference_profile = "us.amazon.nova-pro-v1:0"
foundation_model = inference_profile[3:] 

print(foundation_model)

amazon.nova-pro-v1:0


## 3. Create a Knowledge Base for Amazon Bedrock

These are the steps to complete:
    
1. Create a Knowledge Base Role and its policies
1. Create a Vector Database
1. Create an OpenSearch Index
1. Create a Knowledge Base
1. Create a data source and attach to the recently created Knowledge Base
1. Ingest data to your knowledge Base

### 3.1 Create Knowledge Base Role and Policies

Let's first create IAM policies to allow our Knowledge Base to access Bedrock Titan Embedding Foundation model, Amazon OpenSearch Serverless and the S3 bucket with the Knowledge Base Files.

Once the policies are ready, we will create the Knowledge Base role

In [13]:
# Create IAM policies for KB to invoke embedding model
bedrock_kb_allow_fm_model_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                embedding_model_arn
            ]
        }
    ]
}

kb_bedrock_policy_json = json.dumps(bedrock_kb_allow_fm_model_policy_statement)

kb_bedrock_policy = iam_client.create_policy(
    PolicyName=kb_bedrock_allow_policy_name,
    PolicyDocument=kb_bedrock_policy_json
)

In [14]:
# Create IAM policies for KB to access OpenSearch Serverless
bedrock_kb_allow_aoss_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "aoss:APIAccessAll",
            "Resource": [
                f"arn:aws:aoss:{region}:{account_id}:collection/*"
            ]
        }
    ]
}


kb_aoss_policy_json = json.dumps(bedrock_kb_allow_aoss_policy_statement)

kb_aoss_policy = iam_client.create_policy(
    PolicyName=kb_aoss_allow_policy_name,
    PolicyDocument=kb_aoss_policy_json
)

In [15]:
kb_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowKBAccessDocuments",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}/*",
                f"arn:aws:s3:::{bucket_name}"
            ],
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": f"{account_id}"
                }
            }
        }
    ]
}


kb_s3_json = json.dumps(kb_s3_allow_policy_statement)
kb_s3_policy = iam_client.create_policy(
    PolicyName=kb_s3_allow_policy_name,
    PolicyDocument=kb_s3_json
)

In [16]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
kb_role = iam_client.create_role(
    RoleName=kb_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_aoss_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_s3_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '6917ff65-c224-4fe3-b0b9-9099299de677',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 18 May 2025 07:26:09 GMT',
   'x-amzn-requestid': '6917ff65-c224-4fe3-b0b9-9099299de677',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [17]:
kb_role_arn = kb_role["Role"]["Arn"]
kb_role_arn

'arn:aws:iam::657506130129:role/agent-explorers-insurance-agent-kb-exec-role'

### 3.2 Create Vector Database

Firt of all we have to create a vector store using Amazon OpenSerach Serverless.


In [21]:
# Create OpenSearch Collection
security_policy_json = {
    "Rules": [
        {
            "ResourceType": "collection",
            "Resource":[
                f"collection/{kb_collection_name}"
            ]
        }
    ],
    "AWSOwnedKey": True
}
security_policy = open_search_serverless_client.create_security_policy(
    description='security policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(security_policy_json),
    type='encryption'
)

In [22]:
network_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "dashboard"
      },
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "collection"
      }
    ],
    "AllowFromPublic": True
  }
]

network_policy = open_search_serverless_client.create_security_policy(
    description='network policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(network_policy_json),
    type='network'
)

In [23]:
response = sts_client.get_caller_identity()
current_role = response['Arn']
current_role

'arn:aws:sts::657506130129:assumed-role/AmazonSageMaker-ExecutionRole-20250513T130007/SageMaker'

In [24]:
data_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "Permission": [
          "aoss:DescribeCollectionItems",
          "aoss:CreateCollectionItems",
          "aoss:UpdateCollectionItems",
          "aoss:DeleteCollectionItems"
        ],
        "ResourceType": "collection"
      },
      {
        "Resource": [
          f"index/{kb_collection_name}/*"
        ],
        "Permission": [
            "aoss:CreateIndex",
            "aoss:DeleteIndex",
            "aoss:UpdateIndex",
            "aoss:DescribeIndex",
            "aoss:ReadDocument",
            "aoss:WriteDocument"
        ],
        "ResourceType": "index"
      }
    ],
    "Principal": [
        kb_role_arn,
        f"arn:aws:sts::{account_id}:assumed-role/Admin/*",
        current_role
    ],
    "Description": ""
  }
]

data_policy = open_search_serverless_client.create_access_policy(
    description='data access policy for aoss collection',
    name=kb_collection_name,
    policy=json.dumps(data_policy_json),
    type='data'
)


In [25]:
opensearch_collection_response = open_search_serverless_client.create_collection(
    description='OpenSearch collection for Amazon Bedrock Knowledge Base',
    name=kb_collection_name,
    standbyReplicas='DISABLED',
    type='VECTORSEARCH'
)
opensearch_collection_response

{'createCollectionDetail': {'id': 'qe31vc2xrjmagq96i0ui',
  'name': 'agent-explorers-insurance-kbc',
  'status': 'CREATING',
  'type': 'VECTORSEARCH',
  'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base',
  'arn': 'arn:aws:aoss:us-east-1:657506130129:collection/qe31vc2xrjmagq96i0ui',
  'kmsKeyArn': 'auto',
  'standbyReplicas': 'DISABLED',
  'createdDate': 1747556509722,
  'lastModifiedDate': 1747556509722},
 'ResponseMetadata': {'RequestId': '764a9a82-195c-451d-8669-59435e983834',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '764a9a82-195c-451d-8669-59435e983834',
   'date': 'Sun, 18 May 2025 08:21:49 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '394',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [26]:
collection_arn = opensearch_collection_response["createCollectionDetail"]["arn"]
collection_arn

'arn:aws:aoss:us-east-1:657506130129:collection/qe31vc2xrjmagq96i0ui'

In [27]:
# wait for collection creation
response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    time.sleep(30)
    response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
print('\nCollection successfully created:')
print(response["collectionDetails"])
# Extract the collection endpoint from the response
host = (response['collectionDetails'][0]['collectionEndpoint'])
final_host = host.replace("https://", "")
final_host


Collection successfully created:
[{'id': 'qe31vc2xrjmagq96i0ui', 'name': 'agent-explorers-insurance-kbc', 'status': 'ACTIVE', 'type': 'VECTORSEARCH', 'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base', 'arn': 'arn:aws:aoss:us-east-1:657506130129:collection/qe31vc2xrjmagq96i0ui', 'kmsKeyArn': 'auto', 'standbyReplicas': 'DISABLED', 'createdDate': 1747556509722, 'lastModifiedDate': 1747556532495, 'collectionEndpoint': 'https://qe31vc2xrjmagq96i0ui.us-east-1.aoss.amazonaws.com', 'dashboardEndpoint': 'https://qe31vc2xrjmagq96i0ui.us-east-1.aoss.amazonaws.com/_dashboards'}]


'qe31vc2xrjmagq96i0ui.us-east-1.aoss.amazonaws.com'

### 3.3 - Create OpenSearch Index

Let's now create a vector index to index our data

In [28]:
credentials = boto3.Session().get_credentials()
service = 'aoss'
awsauth = AWS4Auth(
    credentials.access_key, 
    credentials.secret_key,
    region, 
    service, 
    session_token=credentials.token
)

# Build the OpenSearch client
open_search_client = OpenSearch(
    hosts=[{'host': final_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)
# It can take up to a minute for data access rules to be enforced
time.sleep(45)
index_body = {
    "settings": {
        "index.knn": True,
        "number_of_shards": 1,
        "knn.algo_param.ef_search": 512,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {}
    }
}

index_body["mappings"]["properties"][kb_vectorField] = {
    "type": "knn_vector",
    "dimension": 1536,
    "method": {
         "name": "hnsw",
         "engine": "faiss"
    },
}

index_body["mappings"]["properties"][kb_textField] = {
    "type": "text"
}

index_body["mappings"]["properties"][kb_metadataField] = {
    "type": "text"
}

# Create index
response = open_search_client.indices.create(kb_vector_index_name, body=index_body)
print('\nCreating index:')
print(response)

[2025-05-18 08:24:17,845] p2089 {base.py:258} INFO - PUT https://qe31vc2xrjmagq96i0ui.us-east-1.aoss.amazonaws.com:443/bedrock-knowledge-base-index [status:200 request:0.557s]



Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bedrock-knowledge-base-index'}


### 3.5 - Create Knowledge Base
Now that we have the Vector database available in OpenSearch Serverless, let's create a Knowledge Base and associate it with the OpenSearch DB

In [30]:
storage_configuration = {
    'opensearchServerlessConfiguration': {
        'collectionArn': collection_arn, 
        'fieldMapping': {
            'metadataField': kb_metadataField,
            'textField': kb_textField,
            'vectorField': kb_vectorField
        },
        'vectorIndexName': kb_vector_index_name
    },
    'type': 'OPENSEARCH_SERVERLESS'
}

In [32]:
# Creating the knowledge base
try:
    # ensure the index is created and available
    time.sleep(45)
    kb_obj = bedrock_agent_client.create_knowledge_base(
        name=kb_name, 
        description='KB that contains the bedrock documentation',
        roleArn=kb_role_arn,
        knowledgeBaseConfiguration={
            'type': 'VECTOR',  # Corrected type
            'vectorKnowledgeBaseConfiguration': {
                'embeddingModelArn': embedding_model_arn
            }
        },
        storageConfiguration=storage_configuration
    )

    # Pretty print the response
    pprint.pprint(kb_obj)

except Exception as e:
    print(f"Error occurred: {e}")

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '998',
                                      'content-type': 'application/json',
                                      'date': 'Sun, 18 May 2025 08:38:24 GMT',
                                      'x-amz-apigw-id': 'KwVj-HWuIAMEp7g=',
                                      'x-amzn-requestid': 'b98fbb38-0f6c-477f-a145-8ce0e01e317d',
                                      'x-amzn-trace-id': 'Root=1-68299c7f-4c0a1890086d4f99098e55b6'},
                      'HTTPStatusCode': 202,
                      'RequestId': 'b98fbb38-0f6c-477f-a145-8ce0e01e317d',
                      'RetryAttempts': 0},
 'knowledgeBase': {'createdAt': datetime.datetime(2025, 5, 18, 8, 38, 23, 586525, tzinfo=tzlocal()),
                   'description': 'KB that contains the bedrock documentation',
                   'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:657506130129:knowledge-base/VZ4B

In [35]:
# Define the S3 configuration for your data source
s3_configuration = {
    'bucketArn': bucket_arn,
    'inclusionPrefixes': [kb_key]  
}

# Define the data source configuration
data_source_configuration = {
    's3Configuration': s3_configuration,
    'type': 'S3'
}

knowledge_base_id = kb_obj["knowledgeBase"]["knowledgeBaseId"]
knowledge_base_arn = kb_obj["knowledgeBase"]["knowledgeBaseArn"]

chunking_strategy_configuration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# Create the data source
try:
    # ensure that the KB is created and available
    time.sleep(45)
    data_source_response = bedrock_agent_client.create_data_source(
        knowledgeBaseId=knowledge_base_id,
        name=data_source_name,
        description='DataSource for the bedrock documentation',
        dataSourceConfiguration=data_source_configuration,
        vectorIngestionConfiguration = {
            "chunkingConfiguration": chunking_strategy_configuration
        }
    )

    # Pretty print the response
    pprint.pprint(data_source_response)

except Exception as e:
    print(f"Error occurred: {e}")

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '652',
                                      'content-type': 'application/json',
                                      'date': 'Sun, 18 May 2025 08:46:48 GMT',
                                      'x-amz-apigw-id': 'KwWy8FxZIAMEiuA=',
                                      'x-amzn-requestid': 'b0db4d12-75d4-46e5-ba7a-4f05acdec375',
                                      'x-amzn-trace-id': 'Root=1-68299e78-2cad57291317a9a3229c65c0'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'b0db4d12-75d4-46e5-ba7a-4f05acdec375',
                      'RetryAttempts': 0},
 'dataSource': {'createdAt': datetime.datetime(2025, 5, 18, 8, 46, 48, 966326, tzinfo=tzlocal()),
                'dataDeletionPolicy': 'DELETE',
                'dataSourceConfiguration': {'s3Configuration': {'bucketArn': 'arn:aws:s3:::agentexplorer-insurance-data',
           

### 3.6 - Start ingestion job

Once the Knowledge Base and Data Source are created, we can start the ingestion job. During the ingestion job, Knowledge Base will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case Amazon OpenSource Serverless.


In [37]:
# Start an ingestion job
data_source_id = data_source_response["dataSource"]["dataSourceId"]
start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=knowledge_base_id, 
    dataSourceId=data_source_id
)

## 4. Create Agent

We will now create the Agent and associate the Knowledge Base to it. To do so we need to: 
1. Create Agent IAM role and policies
1. Create Agent
1. Associate Agent to Knowledge Base
1. Prepare Agent

### 4.1 - Create Agent IAM role and policies
First we need to create the agent policies that allow bedrock model invocation and Knowledge Base retrieval

In [38]:
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:*::foundation-model/{foundation_model}",
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        },
        {
            "Sid": "AmazonBedrockAgentBedrockGetInferenceProfile",
            "Effect": "Allow",
            "Action":  [
                "bedrock:GetInferenceProfile",
                "bedrock:ListInferenceProfiles",
                "bedrock:UseInferenceProfile"
            ],
            "Resource": "*"
        }

    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [40]:
bedrock_agent_kb_retrival_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve"
            ],
            "Resource": [
                knowledge_base_arn
            ]
        }
    ]
}
bedrock_agent_kb_json = json.dumps(bedrock_agent_kb_retrival_policy_statement)
agent_kb_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_kb_allow_policy_name,
    Description=f"Policy to allow agent to retrieve documents from knowledge base.",
    PolicyDocument=bedrock_agent_kb_json
)

EntityAlreadyExistsException: An error occurred (EntityAlreadyExists) when calling the CreatePolicy operation: A policy called agent-explorers-insurance-agent-kb-allow-policy already exists. Duplicate names are not allowed.

In [41]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)


iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_kb_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '59dd5fa2-cafd-455b-863b-828215edc31f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 19 May 2025 10:13:38 GMT',
   'x-amzn-requestid': '59dd5fa2-cafd-455b-863b-828215edc31f',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

### 4.2 - Create Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the create_agent function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [44]:
# Pause for agent role creation
time.sleep(30)

# Create Agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent supporting Amazon Bedrock Developers.",
    idleSessionTTLInSeconds=1800,
    foundationModel=inference_profile,
    instruction=agent_instruction,
)

time.sleep(30)

Let's now store the agent id in a local variable to use it on the next steps

In [47]:
agent_id = response['agent']['agentId']
agent_id

'M2RWG6DHRU'

### 4.3 - Associate agent to the Knowledge Base
Next, we need to associate the agent created with the Knowledge Base for the Bedrock documentation

In [49]:
agent_kb_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {kb_name} knowledge base to provide accurate responses to the questions about Amazon Bedrock.',
    knowledgeBaseId=knowledge_base_id 
)

### 4.4 - Prepare Agent

Let's create a DRAFT version of the agent that can be used for internal testing.

In [51]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': 'ce9ada3c-d69c-4d8c-9b51-9991278b1420',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 19 May 2025 11:08:16 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ce9ada3c-d69c-4d8c-9b51-9991278b1420',
   'x-amz-apigw-id': 'Kz-dAGnGoAMEvuw=',
   'x-amzn-trace-id': 'Root=1-682b111f-689a4ce1313297f60e654f5b'},
  'RetryAttempts': 0},
 'agentId': 'M2RWG6DHRU',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 5, 19, 11, 8, 16, 121644, tzinfo=tzlocal())}